In [1]:
from src.models.utils.data import FoodPricingDataset
import numpy as np
import torch
from xgboost import DMatrix
from src.models.nlp.pretrained_bert import PreTrainedBERT
from torch.utils.data import DataLoader
from torchvision.models import resnet152
from torchvision.transforms import Compose, Normalize, ToTensor, Resize
from tqdm.notebook import tqdm

In [2]:
def _build_txt_transform():
    pretrained_model_name_or_path = "dbmdz/bert-base-italian-xxl-uncased"
    model_kwargs = {"pretrained_model_name_or_path": pretrained_model_name_or_path}
    return PreTrainedBERT(model_kwargs)


def _build_img_transform(hparams):
    img_dim = hparams["img_dim"]
    transformer = Compose(
        [
            Resize(size=(img_dim, img_dim)),
            ToTensor(),
            Normalize(mean=hparams["img_mean"], std=hparams["img_std"]),
        ]
    )
    module = resnet152(weights="DEFAULT")
    for param in module.parameters():
        param.requires_grad = False
    return lambda img: module(torch.unsqueeze(transformer(img), 0))

In [3]:
hparams = {
    "img_dim": 224,
    "img_mean": [0.485, 0.456, 0.406],
    "img_std": [0.229, 0.224, 0.225],
}

img_transform = _build_img_transform(hparams)
txt_transform = _build_txt_transform()

dataset = FoodPricingDataset(
    img_transform=img_transform,
    # img_transform=lambda x: x,
    txt_transform=txt_transform,
    split="train"
)

Some weights of the model checkpoint at dbmdz/bert-base-italian-xxl-uncased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
INFO:root:Loading the tokenizer with the same name of the model for the PretrainedBERT.


In [4]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=8)

In [5]:
ar = torch.vstack(
    [
        torch.hstack(
            (
                batch["label"],
                torch.squeeze(batch["img"]),
                torch.squeeze(batch["txt"]),
            )
        )
        for batch in tqdm(dataloader)
    ]
).numpy()
labels = ar[:, 0]
features = ar[:, 1:]

  0%|          | 0/155 [00:00<?, ?it/s]

In [10]:
# alternative

language_feature_dim = 768
vision_feature_dim = 1000

batch_size = 32

array = np.zeros((len(dataset), (1 + vision_feature_dim + language_feature_dim)), dtype=np.float32)

for i, batch in enumerate(tqdm(dataloader)):
    array[i * batch_size : i * batch_size + len(batch["label"])] = torch.hstack(
        (
            batch["label"],
            torch.squeeze(batch["img"]),
            torch.squeeze(batch["txt"]),
        )
    ).numpy()


 52%|█████▏    | 81/155 [03:46<03:27,  2.80s/it] 


KeyboardInterrupt: 